In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image

In [3]:
load_dotenv()

True

### Loaded dataset

In [8]:
# Function to load image datasets
def load_image_dataset(image_dir1, image_dir2, image_size=(224, 224)):
    """Loads image datasets from two directories with corresponding labels.

    Args:
        image_dir1: Path to the first directory containing images.
        image_dir2: Path to the second directory containing images.
        image_size: Tuple specifying the desired image size (width, height).

    Returns:
        A tuple containing the image data (NumPy array) and labels (NumPy array).
    """
    images = []
    labels = []

    for filename in os.listdir(image_dir1):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(image_dir1, filename)
            try:
                img = Image.open(filepath).convert("RGB").resize(image_size)
                img_array = np.array(img) / 255.0 # Normalize pixel values
                images.append(img_array)
                labels.append(0) # Label for images from the GOOD cherry
            except Exception as e:
                print(f"Error loading {filename}: {e}")

    for filename in os.listdir(image_dir2):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(image_dir2, filename)
            try:
                img = Image.open(filepath).convert("RGB").resize(image_size)
                img_array = np.array(img) / 255.0
                images.append(img_array)
                labels.append(1) # Label for images from the BAD cherry
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                
    return np.array(images), np.array(labels)

# Function to split dataset
def split_dataset(images, labels, test_size=0.2, val_size=0.1, random_state=42):
    """Splits the image dataset into training, testing, and validation sets."""
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=random_state, stratify=labels)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=val_size / (1 - test_size), random_state=random_state, stratify=y_train)
    return x_train, y_train, x_val, y_val, x_test, y_test


In [9]:
# Example usage (Replace with your actual directories)
image_dir1 = os.getenv('GOOD_DATASET')
image_dir2 = os.getenv('BAD_DATASET')
images, labels = load_image_dataset(image_dir1, image_dir2)
x_train, y_train, x_val, y_val, x_test, y_test = split_dataset(images, labels)

# Print dataset sizes
print("Training set size:", len(x_train))
print("Validation set size:", len(x_val))
print("Testing set size:", len(x_test))

Training set size: 296
Validation set size: 43
Testing set size: 85


### Training model

In [ ]:
# Define the model
def create_model():
    model = keras.Sequential([
        layers.Input(shape=(img_height, img_width, 3)),  # Input shape for RGB images
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(2, activation='sigmoid')  # Output layer for binary classification
    ])
    return model

# Define image dimensions
img_height = 224  # Example dimensions, adjust as needed
img_width = 224

# Create the model
model = create_model()

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define a ModelCheckpoint callback
checkpoint = keras.callbacks.ModelCheckpoint("best_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train the model
model.fit(x_train,
          y_train,
          batch_size=16,
          epochs=10,  # Adjust number of epochs as needed
          validation_data=(x_val, y_val),
          callbacks=[checkpoint])

# Evaluate the model
_, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

2024-11-16 14:07:35.854296: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-11-16 14:07:35.854326: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-16 14:07:35.854333: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-16 14:07:35.854401: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-16 14:07:35.854438: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2024-11-16 14:07:36.439506: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - ETA: 0s - loss: 2.1207 - accuracy: 0.5000
Epoch 1: val_accuracy improved from -inf to 0.69767, saving model to best_model.h5


2024-11-16 14:07:38.031728: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/mlp/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


19/19 [==============================] - 2s 84ms/step - loss: 2.1207 - accuracy: 0.5000 - val_loss: 0.5997 - val_accuracy: 0.6977
Epoch 2/10
18/19 [===========================>..] - ETA: 0s - loss: 0.5281 - accuracy: 0.7465
Epoch 2: val_accuracy improved from 0.69767 to 0.83721, saving model to best_model.h5
19/19 [==============================] - 1s 67ms/step - loss: 0.5265 - accuracy: 0.7500 - val_loss: 0.4215 - val_accuracy: 0.8372
Epoch 3/10
18/19 [===========================>..] - ETA: 0s - loss: 0.4259 - accuracy: 0.8299
Epoch 3: val_accuracy did not improve from 0.83721
19/19 [==============================] - 1s 62ms/step - loss: 0.4176 - accuracy: 0.8345 - val_loss: 0.4769 - val_accuracy: 0.8140
Epoch 4/10
18/19 [===========================>..] - ETA: 0s - loss: 0.3018 - accuracy: 0.8854
Epoch 4: val_accuracy did not improve from 0.83721
19/19 [==============================] - 1s 62ms/step - loss: 0.2988 - accuracy: 0.8851 - val_loss: 0.4097 - val_accuracy: 0.8372
Epoch 5/10

In [25]:
# Confusion matrix
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# import matplotlib.pyplot as plt

# predictions = model.predict(x_test)
# disp = confusion_matrix(y_test, predictions, labels=[0,1])
# disp.plot()
# plt.show()

**Test the best model**

In [34]:
path1 = "./dataset/not_good/20231202_071734_2.jpg" # green cherry
path2 = "./dataset/not_good/20231202_115412_2.jpg" # white cherry
path3 = "./dataset/not_good/20231202_115756_1.jpg" # rotten cherry
path4 = "./dataset/not_good/20231202_114425_2.jpg" # yellow cherry
path5 = "./dataset/not_good/20231202_114447_1.jpg" # half-red cherry
path6 = "bad_cherry.jpg" # green sinjang
path7 = "./dataset/good/20231202_064819.jpg" # red cherry

paths = [path1,path2,path3,path4,path5,path6,path7]
image_size = (224, 224)

def prediction(path):
    # Load model
    model = keras.models.load_model("best_model.h5")

    # Load, convert, and resize the image
    img = Image.open(path).convert("RGB").resize(image_size)
    img_array = np.array(img) / 255.0

    # Add a batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    # print("image size: ", img_array.shape)

    # Predict output
    pred = 1 if model.predict(img_array) > 0.5 else 0
    return pred

for i in range(len(paths)): 
    pred = prediction(paths[i])
    print(f"Predict{i+1}: {pred}")


1/1 [==============================] - 0s 44ms/step


2024-11-16 15:27:02.522140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Predict1: 1
1/1 [==============================] - 0s 44ms/step


2024-11-16 15:27:02.791088: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Predict2: 1
1/1 [==============================] - 0s 44ms/step


...

2024-11-16 15:27:03.564145: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Predict5: 1
1/1 [==============================] - 0s 46ms/step


2024-11-16 15:27:03.810549: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Predict6: 1
1/1 [==============================] - 0s 45ms/step
Predict7: 0


2024-11-16 15:27:04.075699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


**Convert to .tflite**

In [39]:
# Save the model as .tflite
model = keras.models.load_model("best_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/tmp3n0wx5vu/assets


INFO:tensorflow:Assets written to: /var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/tmp3n0wx5vu/assets
2024-11-16 15:30:50.909353: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-11-16 15:30:50.909366: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-11-16 15:30:50.909472: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/tmp3n0wx5vu
2024-11-16 15:30:50.910395: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-11-16 15:30:50.910400: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/tmp3n0wx5vu
2024-11-16 15:30:50.913157: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-11-16 15:30:51.064835: I tensorflow/cc/saved_model/loader.cc:215] Running initialization

### Test with captured image

In [38]:
path1 = "./dataset/not_good/20231202_071734_2.jpg" # green cherry
path2 = "./dataset/not_good/20231202_115412_2.jpg" # white cherry
path3 = "./dataset/not_good/20231202_115756_1.jpg" # rotten cherry
path4 = "./dataset/not_good/20231202_114425_2.jpg" # yellow cherry
path5 = "./dataset/not_good/20231202_114447_1.jpg" # half-red cherry
path6 = "bad_cherry.jpg" # green sinjang
path7 = "./dataset/good/20231202_064819.jpg" # red cherry

paths = [path1,path2,path3,path4,path5,path6,path7]

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# print("input: \n", input_details)
# print("output: \n", output_details)

# Function for making predictions
def predict_tflite(path):
    # Load, convert, and resize the image
    img = Image.open(path).convert("RGB").resize(image_size)
    img_array = np.array(img) / 255.0

    # Add a batch dimension
    img_array = np.expand_dims(img_array, axis=0).astype(np.float32)
    print("image size: ", img_array.shape)


    interpreter.set_tensor(input_details[0]['index'], img_array)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print("model output: ",output_data)
    pred = 1 if output_data > 0.5 else 0
    return pred


# Make the prediction
for i in range(len(paths)):
    pred = predict_tflite(paths[i])
    print(f"Prediction{i+1}: {pred}") # O means good, 1 means bad


image size:  (1, 224, 224, 3)
model output:  [[0.15352581]]
Prediction1: 0
image size:  (1, 224, 224, 3)
model output:  [[0.19350655]]
Prediction2: 0
image size:  (1, 224, 224, 3)
model output:  [[0.25061744]]
Prediction3: 0
image size:  (1, 224, 224, 3)
model output:  [[0.10779668]]
Prediction4: 0
image size:  (1, 224, 224, 3)
model output:  [[0.25800547]]
Prediction5: 0
image size:  (1, 224, 224, 3)
model output:  [[0.56318086]]
Prediction6: 1
image size:  (1, 224, 224, 3)
model output:  [[0.7079757]]
Prediction7: 1
